# 计算 Bimodal Fusion Strategies (CLIP + BERT) 三折交叉验证平均指标

本 notebook 用于计算三次独立运行（不同 fold）的平均指标，包括：
- 总体准确率（Test ACC）
- 每个表情类别的 Precision、Recall、F1-score
- Macro avg 和 Weighted avg 指标

**三折模型路径：**
1. Fold 0: ./data2025/resm/0CLIP_CNNLSTM__0BERT_CNNAttn/fusion_s/
2. Fold 1: ./data2025/resm/1CLIP_CNNLSTM__1BERT_CNNAttn/fusion_s/
3. Fold 2: ./data2025/resm/2CLIP_CNNLSTM__2BERT_CNNAttn/fusion_s/

**融合策略 (6种):**
- concat, weighted_sum, gated, bilinear, cross_attention, tensor

In [ ]:
# ===== GPU 配置 =====
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0，1"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

import gc
import numpy as np
import pickle
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras.layers import Layer, Dense
from keras.regularizers import l2
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print("库导入成功！")
print(f"TensorFlow: {tf.__version__}")

In [ ]:
# ========== 加载数据函数 ==========
def load_features(file_path):
    """加载特征文件"""
    with open(file_path, 'rb') as f:
        return pickle.load(f)

# ========== 自定义层 (用于加载模型) ==========
class CrossAttention(Layer):
    """跨模态注意力层"""
    def __init__(self, units=64, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        
    def build(self, input_shape):
        self.wq = Dense(self.units, kernel_regularizer=l2(0.001))
        self.wk = Dense(self.units, kernel_regularizer=l2(0.001))
        self.wv = Dense(self.units, kernel_regularizer=l2(0.001))
        super().build(input_shape)
        
    def call(self, query, key_value):
        q = self.wq(query)
        k = self.wk(key_value)
        v = self.wv(key_value)
        scores = tf.matmul(q, k, transpose_b=True) / tf.math.sqrt(tf.cast(self.units, tf.float32))
        attn_weights = tf.nn.softmax(scores, axis=-1)
        output = tf.matmul(attn_weights, v)
        return output
    
    def get_config(self):
        config = super().get_config()
        config.update({'units': self.units})
        return config

In [ ]:
# ========== 配置 ==========
# 三折模型路径
fold_configs = [
    {'fold': 0, 'model_dir': './data2025/resm/0CLIP_CNNLSTM__0BERT_CNNAttn/fusion_s/', 'name': 'Fold 0'},
    {'fold': 1, 'model_dir': './data2025/resm/1CLIP_CNNLSTM__1BERT_CNNAttn/fusion_s/', 'name': 'Fold 1'},
    {'fold': 2, 'model_dir': './data2025/resm/2CLIP_CNNLSTM__2BERT_CNNAttn/fusion_s/', 'name': 'Fold 2'}
]

# 融合策略列表 (6种，不含hierarchical)
fusion_methods = ['concat', 'weighted_sum', 'gated', 'bilinear', 'cross_attention', 'tensor']

# 类别标签
class_labels = ['Bored', 'Happy', 'Interested', 'Tired', 'Confused']

# 保存路径
SAVE_DIR = './data2025/reshb3fold/bimodal_fusion'
os.makedirs(SAVE_DIR, exist_ok=True)

print("配置完成！")
print(f"将评估 {len(fold_configs)} 个 fold，每个 fold 有 {len(fusion_methods)} 种融合策略")
print(f"保存路径: {SAVE_DIR}")

In [ ]:
# ========== 对每种融合策略计算三折平均 ==========
all_method_results = {}  # 存储所有融合策略的结果

for method in fusion_methods:
    print("="*70)
    print(f"融合策略: {method.upper()}")
    print("="*70)
    
    method_results = []  # 存储该策略三折的结果
    
    for config in fold_configs:
        fold = config['fold']
        model_path = f"{config['model_dir']}model_{method}.tf"
        
        print(f"\n{config['name']}: {model_path}")
        
        # 加载对应 fold 的测试数据
        clip_data = load_features(f"./data2025/fold{fold}_visual_clip.pkl")
        bert_data = load_features(f"./data2025/fold{fold}_textual_bert.pkl")
        label = load_features(f"./data2025/fold{fold}_labels.pkl")
        
        X_clip_test = np.asarray(clip_data['test'])
        X_bert_test = np.asarray(bert_data['test'])
        y_test = np.asarray(label['test'])
        
        print(f"测试集: CLIP {X_clip_test.shape}, BERT {X_bert_test.shape}")
        print(f"测试标签数量: {len(y_test)}")
        
        # 加载模型
        model = keras.models.load_model(model_path, custom_objects={'CrossAttention': CrossAttention})
        
        # 预测
        pred = model.predict([X_clip_test, X_bert_test], verbose=1)
        predicted_labels = pred.argmax(axis=1)
        
        # 计算准确率
        acc = accuracy_score(y_test, predicted_labels)
        print(f"测试准确率: {acc:.4f}")
        
        # 生成分类报告
        report = classification_report(y_test, predicted_labels,
                                       target_names=class_labels,
                                       digits=4,
                                       zero_division=0,
                                       output_dict=True)
        
        # 保存结果
        method_results.append({
            'name': config['name'],
            'fold': fold,
            'test_acc': acc,
            'report': report,
            'test_labels': y_test.copy(),
            'predictions': predicted_labels.copy()
        })
        
        # 清理内存
        del model, clip_data, bert_data, label
        K.clear_session()
        gc.collect()
    
    all_method_results[method] = method_results
    print(f"\n{method.upper()} 三折评估完成！")

print("\n" + "="*70)
print("所有融合策略评估完成！")
print("="*70)

In [ ]:
# ========== 计算各融合策略的三折平均指标 ==========
print("\n")
print("="*90)
print("Bimodal Fusion Strategies (CLIP + BERT) 三折交叉验证平均结果")
print("="*90)

summary_data = []

for method in fusion_methods:
    results = all_method_results[method]
    
    # 计算平均准确率
    accs = [r['test_acc'] for r in results]
    avg_acc = np.mean(accs)
    std_acc = np.std(accs)
    
    # 计算平均 F1
    f1s = [r['report']['macro avg']['f1-score'] for r in results]
    avg_f1 = np.mean(f1s)
    std_f1 = np.std(f1s)
    
    summary_data.append({
        'Method': method.upper(),
        'Fold0_ACC': accs[0],
        'Fold1_ACC': accs[1],
        'Fold2_ACC': accs[2],
        'Mean_ACC': avg_acc,
        'Std_ACC': std_acc,
        'Mean_F1': avg_f1,
        'Std_F1': std_f1
    })
    
    print(f"\n{method.upper()}:")
    print(f"  Fold 0: {accs[0]:.4f}, Fold 1: {accs[1]:.4f}, Fold 2: {accs[2]:.4f}")
    print(f"  平均 ACC: {avg_acc:.4f} ± {std_acc:.4f}")
    print(f"  平均 F1:  {avg_f1:.4f} ± {std_f1:.4f}")

# 创建汇总DataFrame
summary_df = pd.DataFrame(summary_data)
summary_df = summary_df.sort_values('Mean_ACC', ascending=False)
print("\n" + "="*90)
print("融合策略排名 (按平均ACC排序):")
print(summary_df.to_string(index=False))
print("="*90)

# 保存融合策略汇总到CSV
csv_path1 = f'{SAVE_DIR}/bimodal_fusion_3fold_summary.csv'
summary_df.to_csv(csv_path1, index=False, encoding='utf-8-sig')
print(f"\n融合策略汇总已保存至: {csv_path1}")

In [ ]:
# ========== 计算所有融合策略的类别F1指标 ==========
print("\n【所有融合策略 - 每个表情类别的三折平均F1指标】")
print("-"*100)

# 指定列顺序（6种，无hierarchical）
method_order = ['concat', 'gated', 'tensor', 'weighted_sum', 'bilinear', 'cross_attention']
method_display = ['Concat', 'Gated', 'Tensor', 'Weighted Sum', 'Bilinear', 'Cross-Attn']

# 构建表格数据：行是情感类别，列是融合策略
all_class_f1_data = []

for label in class_labels + ['macro avg', 'weighted avg']:
    row_data = {'类别': label}
    
    for method, display_name in zip(method_order, method_display):
        results = all_method_results[method]
        f1_scores = [r['report'][label]['f1-score'] for r in results]
        avg_f1 = np.mean(f1_scores)
        std_f1 = np.std(f1_scores)
        row_data[display_name] = f"{avg_f1:.4f} ± {std_f1:.4f}"
    
    all_class_f1_data.append(row_data)

# 创建DataFrame
all_class_f1_df = pd.DataFrame(all_class_f1_data)
print(all_class_f1_df.to_string(index=False))

# 保存类别F1指标到CSV（使用utf-8-sig编码确保±符号正确显示）
csv_path2 = f'{SAVE_DIR}/bimodal_fusion_3fold_class_f1_metrics.csv'
all_class_f1_df.to_csv(csv_path2, index=False, encoding='utf-8-sig')
print(f"\n类别F1指标已保存至: {csv_path2}")

In [ ]:
# ========== 可视化 ==========
print("\n正在生成可视化图表...")

# 获取最佳策略
best_method = summary_df.iloc[0]['Method'].lower()
best_results = all_method_results[best_method]

# 计算最佳策略的类别指标
class_metrics = {label: {'f1-score': []} for label in class_labels}
for result in best_results:
    for label in class_labels:
        class_metrics[label]['f1-score'].append(result['report'][label]['f1-score'])

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 子图1: 各融合策略的三折平均准确率
ax1 = axes[0, 0]
methods_sorted = summary_df['Method'].tolist()
mean_accs = summary_df['Mean_ACC'].tolist()
std_accs = summary_df['Std_ACC'].tolist()
colors = ['#2ecc71', '#3498db', '#e74c3c', '#f39c12', '#9b59b6', '#1abc9c']
bars = ax1.bar(methods_sorted, mean_accs, yerr=std_accs, capsize=5, color=colors[:len(methods_sorted)])
ax1.set_ylabel('Test Accuracy', fontsize=12)
ax1.set_title('Bimodal Fusion (CLIP+BERT) - 3-Fold Cross Validation', fontsize=14, fontweight='bold')
ax1.set_ylim([0, 1])
ax1.set_xticklabels(methods_sorted, rotation=30, ha='right')
ax1.grid(True, alpha=0.3)
for bar, acc, std in zip(bars, mean_accs, std_accs):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + std + 0.01,
             f'{acc:.4f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

# 子图2: 各融合策略的三折准确率分布
ax2 = axes[0, 1]
fold_data = []
for method in fusion_methods:
    accs = [r['test_acc'] for r in all_method_results[method]]
    for i, acc in enumerate(accs):
        fold_data.append({'Method': method.upper(), 'Fold': f'Fold {i}', 'ACC': acc})
fold_df = pd.DataFrame(fold_data)
for i, method in enumerate(fusion_methods):
    method_data = fold_df[fold_df['Method'] == method.upper()]
    ax2.scatter([i]*3, method_data['ACC'], s=100, alpha=0.7)
    ax2.plot([i]*3, method_data['ACC'], 'o-', alpha=0.5)
ax2.set_xticks(range(len(fusion_methods)))
ax2.set_xticklabels([m.upper() for m in fusion_methods], rotation=30, ha='right')
ax2.set_ylabel('Test Accuracy', fontsize=12)
ax2.set_title('Accuracy Distribution Across Folds', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

# 子图3: 最佳策略的类别F1分数
ax3 = axes[1, 0]
avg_f1_scores = [np.mean(class_metrics[label]['f1-score']) for label in class_labels]
std_f1_scores = [np.std(class_metrics[label]['f1-score']) for label in class_labels]
bars = ax3.bar(class_labels, avg_f1_scores, yerr=std_f1_scores, capsize=5,
               color=['#3498db', '#2ecc71', '#e74c3c', '#f39c12', '#9b59b6'])
ax3.set_ylabel('F1-Score', fontsize=12)
ax3.set_title(f'Best Method ({best_method.upper()}) - F1 per Category', fontsize=14, fontweight='bold')
ax3.set_ylim([0, 1])
ax3.grid(True, alpha=0.3)

# 子图4: 最佳策略的平均混淆矩阵
ax4 = axes[1, 1]
avg_cm_normalized = np.zeros((len(class_labels), len(class_labels)))
for result in best_results:
    cm = confusion_matrix(result['test_labels'], result['predictions'])
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    avg_cm_normalized += cm_normalized
avg_cm_normalized = avg_cm_normalized / len(best_results)

sns.heatmap(avg_cm_normalized, annot=True, fmt='.3f', cmap='Blues',
            xticklabels=class_labels, yticklabels=class_labels, ax=ax4,
            vmin=0, vmax=1)
ax4.set_xlabel('Predicted', fontsize=12)
ax4.set_ylabel('True', fontsize=12)
ax4.set_title(f'Best Method ({best_method.upper()}) - Avg Confusion Matrix', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/bimodal_fusion_3fold_results.png', dpi=300, bbox_inches='tight')
print(f"图表已保存至: {SAVE_DIR}/bimodal_fusion_3fold_results.png")
plt.show()

In [ ]:
# ========== 最终总结 ==========
print("\n" + "="*80)
print("Bimodal Fusion Strategies (CLIP + BERT) - 三折交叉验证最终总结")
print("="*80)

print("\n【融合策略排名】")
print("-" * 80)
print(f"{'排名':<4}{'策略':<18}{'Fold0':<10}{'Fold1':<10}{'Fold2':<10}{'Mean ACC':<12}{'Std'}")
print("-" * 80)
for rank, (_, row) in enumerate(summary_df.iterrows(), 1):
    print(f"{rank:<4}{row['Method']:<18}{row['Fold0_ACC']:<10.4f}{row['Fold1_ACC']:<10.4f}{row['Fold2_ACC']:<10.4f}{row['Mean_ACC']:<12.4f}{row['Std_ACC']:.4f}")
print("-" * 80)

best_row = summary_df.iloc[0]
print(f"\n【最佳融合策略】: {best_row['Method']}")
print(f"  平均准确率: {best_row['Mean_ACC']:.4f} +/- {best_row['Std_ACC']:.4f}")
print(f"  平均F1分数: {best_row['Mean_F1']:.4f} +/- {best_row['Std_F1']:.4f}")

print(f"\n【保存路径】: {SAVE_DIR}")
print("="*80)
print("分析完成！")
print("="*80)

gc.collect()